In [1]:
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path='.env')

# https://platform.openai.com/api-keys
openai_api_key = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain_community.document_loaders.pdf import PyPDFLoader
loader = PyPDFLoader("./dados/ExplorersGuide.pdf")
pages = loader.load_and_split()

In [ ]:
#!pip install pypdf
#!pip install tiktoken
#!pip install faiss-cpu

In [3]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)


In [4]:
from langchain_community.vectorstores.faiss import FAISS # banco de dados vetorial FAISS
db = FAISS.from_documents(pages, embeddings)


In [ ]:
question = "Qual a cor tradicional da túnica de Link ?"
db.similarity_search(question)[0] # ele acessa o texto e localiza página 36 (primeira é zero)


In [6]:
from langchain_openai import ChatOpenAI
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain_openai import OpenAI
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=openai_api_key)
chain = RetrievalQA.from_llm(llm=llm, retriever=db.as_retriever())
question = "Qual a cor tradicional da túnica de Link ?"
response = chain(question, return_only_outputs=True)
print(response)


C:\Users\otimi\AppData\Local\Temp\ipykernel_21160\2526503213.py:7: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = chain(question, return_only_outputs=True)


{'result': 'A cor tradicional da túnica de Link é verde.'}


In [7]:
# Lendo documentos com o chatGPT
# https://www.youtube.com/watch?v=5tKJjxXr8LQ
# https://github.com/Argonalyst/langchain-exemplos-simples/blob/main/Exemplo%202.ipynb

import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain_openai import ChatOpenAI
import tiktoken

In [8]:
#!pip install textract

import PyPDF2

# Abrir o arquivo PDF
with open("dados/bitcoin.pdf", "rb") as file:
    reader = PyPDF2.PdfReader(file)
    num_pages = len(reader.pages)
    
    all_text = ""
    
    for page_num in range(num_pages):
        page = reader.pages[page_num]
        text = page.extract_text()
        if text:
            all_text += text
    
    # Salvar todo o texto em um arquivo
    with open("dados/bitcoin.txt", "w", encoding="utf-8") as output_file:
        output_file.write(all_text)


In [9]:
#import textract 
#doc = textract.process("bitcoin.pdf") # deu erro !

#with open('bitcoin.txt', 'w', encoding="utf-8") as f:
#    f.write(doc.decode('utf-8'))

import os
os.environ['CURL_CA_BUNDLE'] = ''

with open('bitcoin.txt', 'r', encoding="utf-8") as f:
    text = f.read()
    
#tokenizer = GPT2TokenizerFast.from_pretrained("gpt2", use_ssl=False) 
tokenizer = tiktoken.get_encoding("cl100k_base")

def count_tokens(text: str) -> int:
    return len(tokenizer.encode(text))

text_splitter = RecursiveCharacterTextSplitter( # divide o PDF em blocos/chunks de 512 tokens
    chunk_size = 512,
    chunk_overlap  = 24,
    length_function = count_tokens,
)

chunks = text_splitter.create_documents([text])

In [10]:
# https://platform.openai.com/docs/guides/embeddings/embedding-model
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key, model="text-embedding-3-small")
db = FAISS.from_documents(chunks, embeddings)

In [11]:
query = "O que é uma moeda eletrônica?"
docs = db.similarity_search(query)
len(docs)

4

In [12]:
docs[0].page_content

'Bitcoin: Um Sistema de Dinheiro Eletrônico Peer-to-Peer\nSatoshi Nakamoto\nsatoshin@gmx.com\nwww.bitcoin.org\nTraduzido para Português de bitcoin.org/bitcoin.pdf\npor Rodrigo Silva Pinto - http://linkedin.com/in/rodrigosilvap\nResumo. Uma versão puramente peer-to-peer de dinheiro eletrônico permitiria que\npagamentos  on-line  fossem enviados  diretamente  de uma parte  para  outra,  sem\npassar por uma instituição financeira. As assinaturas digitais fornecem parte da\nsolução, mas os principais benefícios são perdidos se um terceiro confiável ainda é\nnecessário para evitar o gasto duplo. Nós propomos uma solução para o problema de\ngasto duplo usando uma rede peer-to-peer. A rede insere data e hora nas transações\natravés de um hash em uma cadeia contínua de prova-de-trabalho à base de hash,\nformando um registro que não pode ser alterado sem refazer a prova-de-trabalho. A\ncadeia mais longa não só serve como prova da seqüência de eventos testemunhados,\nmas prova de que ela veio do

In [13]:
docs[1].page_content

'grupo cooperado de nós atacantes.\n2.    Transações\nNós definimos uma moeda eletrônica como uma cadeia de assinaturas digitais. Cada proprietário\ntransfere a moeda para o seguinte por uma assinatura digital de hash da operação anterior e a\n1chave pública do dono da próxima e adicionando-os ao fim da moeda. Um sacador pode verificar\nas assinaturas para verificar a cadeia de propriedade.\nO problema, claro, é o sacador não poder confirmar se um dos pagadores não fez gasto duplo\nda moeda. Uma solução comum é a introdução de uma autoridade central confiável, ou casa da\nmoeda, que verifique o gasto duplo para todas as transações. Depois de cada transação, a moeda\ndeve ser devolvida à casa da moeda para a emissão de uma nova, e apenas moedas emitidas\ndiretamente da casa da moeda são confiáveis de não ser gastas duplamente. O problema desta\nsolução é que o destino de todo o sistema monetário depende da empresa que gerencia a casa da\nmoeda, com todas as transações tendo de passar po

In [14]:
docs[2].page_content

'baseado em confiança. Transações completamente não-reversíveis não são possíveis, uma vez\nque as instituições financeiras não podem evitar a mediação de conflitos. O custo da mediação\naumenta os custos de transação, o que limita o tamanho mínimo prático da transação e elimina a\npossibilidade  de  pequenas  transações  ocasionais,  e  há  um  custo  mais  amplo  na  perda  da\ncapacidade  de  fazer  pagamentos  não  reversível  para  serviços  não  reversíveis.  Com  a\npossibilidade  de  reversão,  a  necessidade  de  confiança  se  espalha.  Comerciantes  devem  ser\ncautelosos com os seus clientes, incomodando-os para obter mais informações do que seria de\noutra forma necessária. Uma certa percentagem de fraude é aceita como inevitável. Estes custos e\nincertezas  de  pagamento  podem ser  evitados  ao  vivo  usando  moeda  física,  mas  não  existe\nnenhum mecanismo para fazer pagamentos ao longo de um canal de comunicação sem uma parte\nconfiável.\nO que é necessário é um sist

In [16]:
docs[3].page_content

'#include <math.h>double AttackerSuccessProbability(double q, int z){    double p = 1.0 - q;    double lambda = z * (q / p);    double sum = 1.0;    int i, k;    for (k = 0; k <= z; k++)    {        double poisson = exp(-lambda);        for (i = 1; i <= k; i++)            poisson *= lambda / i;        sum -= poisson * (1 - pow(q / p, z - k));    }    return sum;}Resolvendo para P menor que 0,1%...\n12.    Conclusão\nPropusemos um sistema de transações eletrônicas, sem depender de confiança. Começamos com\no framework habitual de moedas feitas de assinaturas digitais, que fornece um forte controle de\npropriedade, mas é incompleto sem uma maneira de evitar o gasto duplo. Para resolver isso,\npropusemos uma rede peer-to-peer usando prova de trabalho para gravar um histórico público de\ntransações que rapidamente se torna computacionalmente impraticável para um atacante para\nmudar se nós honestos  controlarem a maioria do poder de CPU. A rede é robusta em sua\nsimplicidade não estruturad

In [17]:
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=openai_api_key)
chain = load_qa_chain(llm, chain_type="stuff")
chain.run(input_documents=docs, question=query)

C:\Users\otimi\AppData\Local\Temp\ipykernel_21160\3252312837.py:2: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/v0.2/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/v0.2/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/v0.2/docs/how_to/#qa-with-rag
  chain = load_qa_chain(llm, chain_type="stuff")
C:\Users\otimi\AppData\Local\Temp\ipykernel_21160\3252312837.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  chain.run(input_documents=docs, question=query)


'Uma moeda eletrônica é definida como uma cadeia de assinaturas digitais. Cada proprietário transfere a moeda para o próximo por meio de uma assinatura digital do hash da operação anterior e da chave pública do próximo proprietário, adicionando essas informações ao final da moeda. A moeda eletrônica permite que pagamentos online sejam enviados diretamente de uma parte para outra sem a necessidade de uma instituição financeira como terceiro confiável.'